In [1]:
from send_to_query import *
BUCKET_NAME = 'edit-data-eng-project-group3'  # Replace with your bucket name
SOURCE_JSON_FILE = 'calendar_dates.csv'  # Path to the JSON file in the bucket
BIGQUERY_PROJECT = 'data-eng-dev-437916'  # Replace with your project ID
BIGQUERY_DATASET = 'data_eng_project_group3'  # Replace with your BigQuery dataset name
BIGQUERY_TABLE = 'lines_test'  # Replace with your BigQuery table name

import pandas as pd
from google.api_core.exceptions import NotFound
from send_to_query import unhashable_types_to_text, restore_unhashable_types




In [2]:
try:
    client = bigquery.Client(project=BIGQUERY_PROJECT)
    table_id = f"{BIGQUERY_PROJECT}.{BIGQUERY_DATASET}.{BIGQUERY_TABLE}"
    existing_table = client.list_rows(table_id).to_dataframe() 
except NotFound as error:
    print("Table does not exist yet, proceeding with sending all data.")



/Users/ineslopes/Documents/de_edit/CarrisInsight/.project/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/ineslopes/Documents/de_edit/CarrisInsight/.project/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [3]:
existing_table.head()

,color,facilities,id,localities,long_name,municipalities,patterns,routes,short_name,text_color
0,#0C807E,[],2650,"[Loures, Odivelas, Costa da Caparica]",Cidade Nova - Costa da Caparica,"[1107, 1116, 1503]","[2650_0_1, 2650_0_2]",[2650_0],2650,#FFFFFF
1,#0C807E,[],2650,"[Loures, Odivelas, Costa da Caparica]",Cidade Nova - Costa da Caparica,"[1107, 1116, 1503]","[2650_0_1, 2650_0_2]",[2650_0],2650,#FFFFFF
2,#0C807E,[],2650,"[Loures, Odivelas, Costa da Caparica]",Cidade Nova - Costa da Caparica,"[1107, 1116, 1503]","[2650_0_1, 2650_0_2]",[2650_0],2650,#FFFFFF
3,#0C807E,[],2651,"[Loures, Bucelas, Pontinha, Vila Franca de Xir...",Bucelas - Costa da Caparica (Terminal),"[1107, 1114, 1503]","[2651_0_1, 2651_0_2]",[2651_0],2651,#FFFFFF
4,#0C807E,[],2651,"[Loures, Bucelas, Pontinha, Vila Franca de Xir...",Bucelas - Costa da Caparica (Terminal),"[1107, 1114, 1503]","[2651_0_1, 2651_0_2]",[2651_0],2651,#FFFFFF


In [8]:
from io import StringIO
csv_buffer = StringIO(dataframe)
df = pd.read_csv(csv_buffer)

In [4]:
#table form bucket
dataframe = read_json_from_gcs(BUCKET_NAME, SOURCE_JSON_FILE)


/Users/ineslopes/Documents/de_edit/CarrisInsight/.project/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/ineslopes/Documents/de_edit/CarrisInsight/.project/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
2025-01-09 00:48:12,627 - INFO - Read json from bucket a

In [9]:
df

,date,day_type,exception_type,holiday,period,service_id
0,20240101,3,1,1,2,26_MPLKH
1,20240102,1,1,0,2,4_MPLKH
2,20240103,1,1,0,1,1_MPLKH
3,20240104,1,1,0,1,1_MPLKH
4,20240105,1,1,0,1,1_MPLKH
...,...,...,...,...,...,...
2493,20251229,1,1,0,2,12_5ZVUM
2494,20251230,1,1,0,2,12_5ZVUM
2495,20251230,1,1,0,2,1202_5ZVUM
2496,20251231,1,1,0,2,12_5ZVUM


In [82]:
merged_df = (
    pd.merge(
        dataframe
        .map(unhashable_types_to_text),
        existing_table.map(unhashable_types_to_text),
        on=existing_table.columns.to_list(),
        how='right',
        indicator=True)
        )
merged_df.head()

,color,facilities,id,localities,long_name,municipalities,patterns,routes,short_name,text_color,_merge
0,#0C807E,[],2650,['Loures' 'Odivelas' 'Costa da Caparica'],Cidade Nova - Costa da Caparica,['1107' '1116' '1503'],['2650_0_1' '2650_0_2'],['2650_0'],2650,#FFFFFF,right_only
1,#0C807E,[],2650,['Loures' 'Odivelas' 'Costa da Caparica'],Cidade Nova - Costa da Caparica,['1107' '1116' '1503'],['2650_0_1' '2650_0_2'],['2650_0'],2650,#FFFFFF,right_only
2,#0C807E,[],2650,['Loures' 'Odivelas' 'Costa da Caparica'],Cidade Nova - Costa da Caparica,['1107' '1116' '1503'],['2650_0_1' '2650_0_2'],['2650_0'],2650,#FFFFFF,right_only
3,#0C807E,[],2651,['Loures' 'Bucelas' 'Pontinha' 'Vila Franca de...,Bucelas - Costa da Caparica (Terminal),['1107' '1114' '1503'],['2651_0_1' '2651_0_2'],['2651_0'],2651,#FFFFFF,right_only
4,#0C807E,[],2651,['Loures' 'Bucelas' 'Pontinha' 'Vila Franca de...,Bucelas - Costa da Caparica (Terminal),['1107' '1114' '1503'],['2651_0_1' '2651_0_2'],['2651_0'],2651,#FFFFFF,right_only


In [83]:
columns = existing_table.columns.to_list()

In [84]:
upload_df=(
        merged_df
        .query('_merge == "left_only"')
        .drop(columns='_merge')
        .map(restore_unhashable_types)
        )

In [81]:
upload_df

,color,facilities,id,localities,long_name,municipalities,patterns,routes,short_name,text_color


In [85]:
def clean_none_list(col):
    if type(col[0]) == list:
        return  col.apply( lambda row: [item for item in row if item != None])
    else: 
        return col